In [ ]:
import pandas as pd
import neattext.functions as nfs
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import time

start = time.time()
df=pd.read_csv(r"/content/drive/MyDrive/final_project/Emotion_project.csv")
df = df.dropna() # drop columns with NA values
df["clean_text"] = df["Text"].apply(nfs.remove_stopwords) # User handles and remove Stopwords

X = df["clean_text"] # Extract two Features from the data
y = df["Emotion"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100) # Let's split the data into two parts

lr = Pipeline(steps=[("cv",CountVectorizer()),("lr",LogisticRegression(max_iter=10))]) # Create a Model on LogisticRegression
lr.fit(X_train,y_train)

end = time.time()
print("Process time: ",end - start)

print("LR Accuracy: ",lr.score(X_test,y_test)) # Show the Score of LogisticRegression Model

text = input("Input any text: ")
label = lr.predict([text])
prob = lr.predict_proba([text])
print("Label: ", label)
print(prob)

In [ ]:
import matplotlib.colors as mcolors
import numpy as np

!pip3 install ColabTurtle
from ColabTurtle.Turtle import *

def secondmax(list):
  sublist = [x for x in list if x < list.max()]
  indices = np.where(list == max(sublist))
  return max(sublist), indices[0][0]

def transform(list, text) :
    # anger / fear / happy / sadness / surprize
    emotions = [[193, 73, 83], [131, 140, 111], [232, 209, 155], [36, 128, 190], [228, 176, 165]]
    rgb = []
    for i in range(0, 3):
        weight = 0
        for j in range(0, 5):
            alpha = 1
            if list[j] == max(list): alpha = 1.2  # first
            elif list[j] == secondmax(list)[0]: alpha = 1.2  # second
            elif list[j] == min(list): alpha = 0.8  # last
            weight += emotions[j][i] * list[j] * alpha
        weight /= 255
        if weight > 1: weight = 1
        rgb.append(weight)

    (h, s, v) = mcolors.rgb_to_hsv(rgb)      
    light = mcolors.hsv_to_rgb([h, 0.05, 0.9])
    dark = mcolors.hsv_to_rgb([h, s, 0.5])

    percentages = []
    for i in list :
        i *= 1000
        i = int(i)
        i /= 10
        percentages.append(' ' + str(i) + ' %')
    color_dict = {
        list[0]: "Anger",
        list[1]: "Fear",
        list[2]: "Happy",
        list[3]: "Sadness",
        list[4]: "Surprize"
    }
    first = list.argmax()
    second = secondmax(list)[1]

    initializeTurtle()
    penup()
    hideturtle()
    bgcolor("#FFFFFF")
    color(mcolors.to_hex(dark))
    style = (20, 'Verdana', 'italic')
    write(text, align='center', font=style)
    backward(50)
    color(mcolors.to_hex(rgb))
    style = (20, 'Verdana', 'bold')
    write(color_dict[list[first]] + percentages[first] + ' + ' + color_dict[list[second]] + percentages[second], align='center', font=style)

transform(prob[0],text)